In [29]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math

import treasureArea as ta
import webCamera as wc

class blockDetect:
    def __init__(self):
        self.treasureArea = ta.TreasureArea()
        self.webCamera = wc.WebCamera(0)

        #閾値
        self.hsvRanges = [
            ((357, 55, 50), (360, 100, 100)),   # red
            ((0, 55, 50), (17, 100, 100)),      # Red-ish colors H=7
            ((37, 50, 25), (57, 100, 100)),     # Yellow-ish colors H=47
            ((120, 15, 10), (160, 100, 100)),   # Green H=150
            ((197, 50, 25), (217, 100, 100))    # blue H=207
        ]
        #閾値サイズ
        self.sizeRange = (1000, 80000)
    
    def detectArea(self, imagePath:str):
        image = cv2.imread(imagePath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        maskImage = self.__imageProcess(image, self.hsvRanges)
        points = self.__findPoint(maskImage, self.sizeRange)
        areaPoints = self.__findArea(points)
        
        filtered_image = np.zeros_like(maskImage)
        centroid_image = cv2.cvtColor(filtered_image, cv2.COLOR_GRAY2RGB)
        for x in points:
            #print(x)
            cv2.circle(centroid_image, (x[0], x[1]), 10, (255, 0, 0), -1)
        cv2.imwrite("result_image.png", centroid_image)

    def detectBlock(self):
        pass
    
    def __imageProcess(self, image, hsv_ranges):
        """
        画像をhsv_rangesの閾値を用いて処理する。\n
        return:画像
        """
        # Resize the image to x% of its original size
        height, width, _ = image.shape
        new_width = int(width * 1.0)
        new_height = int(height * 1.0)
        resized_image = cv2.resize(image, (new_width, new_height))

        # Convert the resized image to HSV color space
        hsv_image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2HSV)

        # Convert the provided HSV ranges to OpenCV format
        converted_ranges = [
            (
                tuple(map(int, np.round(np.multiply(min_HSV, np.array([180/360, 255/100, 255/100]))))),
                tuple(map(int, np.round(np.multiply(max_HSV, np.array([180/360, 255/100, 255/100]))))
            )) for min_HSV, max_HSV in hsv_ranges
        ]

        # List to store binary masks for each converted HSV range
        binary_masks = []

        # Iterate over the converted HSV ranges and create binary masks
        for (min_HSV, max_HSV) in converted_ranges:
            mask = cv2.inRange(hsv_image, np.array(min_HSV), np.array(max_HSV))
            binary_masks.append(mask)

        # Combine all binary masks
        combined_mask = np.zeros_like(binary_masks[0])
        for mask in binary_masks:
            combined_mask = cv2.bitwise_or(combined_mask, mask)

        return combined_mask    

    def __findPoint(self, image, sizeRange):
        """
        マスク処理した画像を利用し、sizeRange間の閾値で重心を出力する。\n
        return:重心配列
        """

        # Load the image
        #image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Threshold the image to binary using Otsu's method
        _, binarized = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Find contours
        contours, _ = cv2.findContours(binarized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create an all zeros image and draw the contours filled with 1
        filled_image = np.zeros_like(image)
        cv2.drawContours(filled_image, contours, -1, (1), thickness=cv2.FILLED)

        # Find contours for the "Contours Filled with 1" image
        contours_filled, _ = cv2.findContours(filled_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filter contours based on area and fill them with 1, others with 0
        filtered_image = np.zeros_like(filled_image)
        for contour in contours_filled:
            if 1000 <= cv2.contourArea(contour) <= 80000:
                cv2.drawContours(filtered_image, [contour], -1, (1), thickness=cv2.FILLED)

        # Calculate and mark the centroids for each contour
        retPoints = []
        for contour in contours_filled:
            if sizeRange[0] <= cv2.contourArea(contour) <= sizeRange[1]:
                # Calculate moments for each contour
                M = cv2.moments(contour)

                # Calculate x,y coordinate of centroid
                if M["m00"] != 0:  # Avoid division by zero
                    cX = int(M["m10"] / M["m00"])
                    cY = int(M["m01"] / M["m00"])
                else:
                    cX, cY = 0, 0

                retPoints.append([cX, cY])    
        return retPoints

    def __findArea(self, points):
        """
        ポイント座標からエリアを特定する関数\n
        return:エリアを構成するポインタ配列
        """
        for i in points:
            degrees = []
            for j in points:
                if not(i == j):
                    dx = i[0] - j[0]
                    dy = i[1] - j[1]

                    # アークタンジェントを使用して角度を計算（ラジアン）
                    angle_radians = math.atan2(dy, dx)

                    # ラジアンから度に変換
                    angle_degrees = math.degrees(angle_radians)
                    degrees.append(angle_degrees)
                    # 角度を出力
                    #print(f'{i} -> {j} 角度（度）: {angle_degrees}')
            degrees.sort()
            for x in degrees:
                print(f'角度（度）: {x}')
            print("\n")

    def __findBlock(self, area):
        pass


In [30]:
if __name__ == "__main__":
    blockDetect = blockDetect()
    blockDetect.detectArea("./image.jpg")

角度（度）: 70.20406838410193
角度（度）: 70.33965987470388
角度（度）: 70.40771810894847
角度（度）: 72.11938143644569
角度（度）: 97.5699511707092
角度（度）: 102.54166005965669
角度（度）: 107.61762315095817
角度（度）: 115.34918312084034
角度（度）: 125.96051719664487
角度（度）: 125.99955314960455
角度（度）: 126.12944414324086
角度（度）: 129.7769518241593
角度（度）: 134.96407788588976
角度（度）: 140.24027774791895
角度（度）: 145.57951813420019
角度（度）: 149.94755228881016
角度（度）: 157.23141766180836
角度（度）: 157.41531700944122
角度（度）: 157.81598665473004
角度（度）: 176.7043690012946


角度（度）: -3.2956309987054166
角度（度）: 11.76323802731335
角度（度）: 16.399627619056105
角度（度）: 18.746466093604614
角度（度）: 22.650075547035566
角度（度）: 26.300407426233928
角度（度）: 27.852043938897854
角度（度）: 29.67259142318284
角度（度）: 31.32708713439522
角度（度）: 35.05942696688698
角度（度）: 40.47652784109833
角度（度）: 41.51548211705574
角度（度）: 43.52904498498679
角度（度）: 47.64762966529222
角度（度）: 52.80899246029574
角度（度）: 59.047834764453164
角度（度）: 62.059113819258364
角度（度）: 71.56505117707799
角度（度）: 102.18238959692225
角